<a href="https://colab.research.google.com/github/tibayo1/Sound-of-AI/blob/main/Making_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

In [2]:
##!cp /content/drive/MyDrive/Colab_Notebooks/Sound_of_AI_Neural.py
#import torch
#from train import FeedForwardNet

class_mapping = [
         "0", "1", "2", "3", "4", "5", "6", "7", "8", "9"       
  ]



In [3]:
def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
  for inputs, targets in data_loader:
    inputs,targets = inputs.to(device), targets.to(device)

    #calulate loss
    pred = model(inputs)
    loss = loss_fn(pred, targets)

    #backpropagate loss and update weights
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
  print(f"Loss: {loss.item()}")

def train(model, data_loader, loss_fn, optimiser, device, epochs):
  for i in range (epochs):
    print(f"Epoch {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimiser, device)
    print("-------------------------------")
  print("Training is done")

def predict(model, input, target, class_mapping):
  model.eval()
  with torch.no_grad():
    predictions = model(input)
    #Tensor {}

    predicted_index = predictions[0].argmax(0)
    predicted = class_mapping[predicted_index]
    expected = class_mapping[target]
  return predicted, expected

In [4]:
class FeedForwardNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.dense_layers = nn.Sequential (
        nn.Linear(28*28, 256),
        nn.ReLU(),
        nn.Linear(256,10)
    )
    self.softmax = nn.Softmax(dim=1)
  def forward(self, input_data):
    f_data = self.flatten(input_data)
    logits = self.dense_layers(f_data)
    pred = self.softmax(logits)
    return pred

In [5]:
if __name__ == "__main__":
  feed_forward_net = FeedForwardNet()
  state_dict = torch.load('feedforwardnet.pth')
  feed_forward_net.load_state_dict(state_dict)

  validation_data = DataLoader(
    datasets.MNIST('../data', train=False, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True)
  input,target = validation_data[0][0],validation_data[0][1] 

  #make an inference
  predicted, expected = predict(feed_forward_net,inputs, target, class_mapping)


  print(f"Predicted:'{predicted}', expected: '{expected}' ")

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


TypeError: ignored